In [10]:
#!/usr/bin/env python3
"""
IPv4/IPv6 Fragmentation Demo - No Root Required
This version creates and analyzes fragments without actually sending them
"""

from scapy.all import *

class FragmentationTester:
    def __init__(self, target_ip, interface="eth0"):
        self.target_ip = target_ip
        self.interface = interface
        self.results = []
        
    def test_ipv4_fragmentation(self, payload_sizes=[1500, 3000, 5000]):
        """
        Test IPv4 fragmentation with different payload sizes
        MTU = 1500 bytes (standard)
        """
        print("\n=== Testing IPv4 Fragmentation ===")
        
        for size in payload_sizes:
            print(f"\nCreating IPv4 packet with payload size: {size} bytes")
            
            # Create payload
            payload = Raw(load="A" * size)
            
            # Create IP packet
            pkt = IP(dst=self.target_ip)/ICMP()/payload
            
            print(f"  Original packet size: {len(pkt)} bytes")
            
            # Fragment the packet (MTU=1500, fragsize=1480 for 20-byte header)
            frags = fragment(pkt, fragsize=1480)
            print(f"  Packet fragmented into {len(frags)} fragments")
            
            # Analyze fragments
            for i, frag in enumerate(frags, 1):
                if IP in frag:
                    ip_layer = frag[IP]
                    print(f"    Fragment {i}:")
                    print(f"      - ID: {ip_layer.id}")
                    print(f"      - Flags: MF={int(ip_layer.flags.MF)}, DF={int(ip_layer.flags.DF)}")
                    print(f"      - Fragment Offset: {ip_layer.frag} (= {ip_layer.frag * 8} bytes)")
                    print(f"      - Total Length: {ip_layer.len} bytes")
                    print(f"      - More Fragments: {'Yes' if ip_layer.flags.MF else 'No'}")
            
            self.results.append({
                'protocol': 'IPv4',
                'size': size,
                'fragments': len(frags),
                'packets': frags
            })
    
    def test_ipv6_fragmentation(self, payload_sizes=[1500, 3000, 5000]):
        """
        Test IPv6 fragmentation with different payload sizes
        Minimum MTU = 1280 bytes
        """
        print("\n=== Testing IPv6 Fragmentation ===")
        
        for size in payload_sizes:
            print(f"\nCreating IPv6 packet with payload size: {size} bytes")
            
            # Create payload
            payload = Raw(load="B" * size)
            
            # Create IPv6 packet
            pkt = IPv6(dst=self.target_ip)/ICMPv6EchoRequest()/payload
            
            print(f"  Original packet size: {len(pkt)} bytes")
            
            # Fragment the packet (fragSize=1280 for minimum MTU)
            frags = fragment6(pkt, fragSize=1280)
            print(f"  Packet fragmented into {len(frags)} fragments")
            
            # Analyze fragments
            for i, frag in enumerate(frags, 1):
                if IPv6 in frag:
                    print(f"    Fragment {i}:")
                    print(f"      - Next Header: {frag[IPv6].nh}")
                    print(f"      - Payload Length: {frag[IPv6].plen}")
                    
                    if IPv6ExtHdrFragment in frag:
                        frag_hdr = frag[IPv6ExtHdrFragment]
                        print(f"      - Fragment ID: 0x{frag_hdr.id:08x}")
                        print(f"      - Fragment Offset: {frag_hdr.offset} (= {frag_hdr.offset * 8} bytes)")
                        print(f"      - More Fragments: {'Yes' if frag_hdr.m else 'No'}")
            
            self.results.append({
                'protocol': 'IPv6',
                'size': size,
                'fragments': len(frags),
                'packets': frags
            })
    
    def generate_report(self):
        """Generate a summary report of the tests"""
        print("\n" + "="*60)
        print("FRAGMENTATION TEST SUMMARY")
        print("="*60)
        
        print("\nProtocol | Payload Size | Number of Fragments")
        print("-" * 60)
        
        for result in self.results:
            print(f"{result['protocol']:8} | {result['size']:12} | {result['fragments']}")
        
        print("\n" + "="*60)
    
    def compare_protocols(self):
        """Compare IPv4 and IPv6 fragmentation behavior"""
        print("\n=== Protocol Comparison ===")
        print("\nIPv4 Fragmentation:")
        print("  - Can be done by routers")
        print("  - Uses 13-bit fragment offset (8-byte units)")
        print("  - 16-bit Identification field in main header")
        print("  - DF (Don't Fragment) flag available")
        print("  - Header size: 20 bytes (minimum)")
        
        print("\nIPv6 Fragmentation:")
        print("  - Only source node can fragment")
        print("  - Uses Fragment Extension Header (8 bytes)")
        print("  - 32-bit Identification field")
        print("  - Minimum MTU: 1280 bytes")
        print("  - Base header size: 40 bytes")
        print("  - More efficient reassembly (only at destination)")
    
    def export_pcap(self, filename="fragments.pcap"):
        """Export all fragments to a PCAP file"""
        all_packets = []
        for result in self.results:
            all_packets.extend(result['packets'])
        
        if all_packets:
            wrpcap(filename, all_packets)
            print(f"\n✓ Exported {len(all_packets)} packets to {filename}")
            print(f"  You can analyze this file with Wireshark")
        else:
            print("\n✗ No packets to export")


# Example usage (no root required - creates but doesn't send packets)
def demo():
    """Demo function - creates fragments without sending"""
    print("="*60)
    print("IPv4/IPv6 FRAGMENTATION DEMO")
    print("(Analysis mode - no packets sent, no root required)")
    print("="*60)
    
    # IPv4 test
    print("\n### Testing IPv4 ###")
    tester_v4 = FragmentationTester("192.168.1.100")
    tester_v4.test_ipv4_fragmentation([1400, 3000, 5000])
    
    # IPv6 test
    print("\n### Testing IPv6 ###")
    tester_v6 = FragmentationTester("2001:db8::1")
    tester_v6.test_ipv6_fragmentation([1400, 3000, 5000])
    
    # Reports
    print("\n### IPv4 Results ###")
    tester_v4.generate_report()
    
    print("\n### IPv6 Results ###")
    tester_v6.generate_report()
    
    # Comparison
    tester_v4.compare_protocols()
    
    # Export to PCAP
    tester_v4.export_pcap("ipv4_fragments.pcap")
    tester_v6.export_pcap("ipv6_fragments.pcap")
    
    print("\n✓ Demo complete!")

# Uncomment to run the demo
demo()


IPv4/IPv6 FRAGMENTATION DEMO
(Analysis mode - no packets sent, no root required)

### Testing IPv4 ###

=== Testing IPv4 Fragmentation ===

Creating IPv4 packet with payload size: 1400 bytes
  Original packet size: 1428 bytes
  Packet fragmented into 1 fragments
    Fragment 1:
      - ID: 1
      - Flags: MF=0, DF=0
      - Fragment Offset: 0 (= 0 bytes)
      - Total Length: None bytes
      - More Fragments: No

Creating IPv4 packet with payload size: 3000 bytes
  Original packet size: 3028 bytes
  Packet fragmented into 3 fragments
    Fragment 1:
      - ID: 1
      - Flags: MF=1, DF=0
      - Fragment Offset: 0 (= 0 bytes)
      - Total Length: None bytes
      - More Fragments: Yes
    Fragment 2:
      - ID: 1
      - Flags: MF=1, DF=0
      - Fragment Offset: 185 (= 1480 bytes)
      - Total Length: None bytes
      - More Fragments: Yes
    Fragment 3:
      - ID: 1
      - Flags: MF=0, DF=0
      - Fragment Offset: 370 (= 2960 bytes)
      - Total Length: None bytes
      - 

  Original packet size: 1448 bytes


  Packet fragmented into 2 fragments
    Fragment 1:
      - Next Header: 44
      - Payload Length: None
      - Fragment ID: 0x9ceff89d
      - Fragment Offset: 0 (= 0 bytes)
      - More Fragments: Yes
    Fragment 2:
      - Next Header: 44
      - Payload Length: None
      - Fragment ID: 0x9ceff89d
      - Fragment Offset: 154 (= 1232 bytes)
      - More Fragments: No

Creating IPv6 packet with payload size: 3000 bytes
  Original packet size: 3048 bytes
  Packet fragmented into 3 fragments
    Fragment 1:
      - Next Header: 44
      - Payload Length: None
      - Fragment ID: 0xb159b8aa
      - Fragment Offset: 0 (= 0 bytes)
      - More Fragments: Yes
    Fragment 2:
      - Next Header: 44
      - Payload Length: None
      - Fragment ID: 0xb159b8aa
      - Fragment Offset: 154 (= 1232 bytes)
      - More Fragments: Yes
    Fragment 3:
      - Next Header: 44
      - Payload Length: None
      - Fragment ID: 0xb159b8aa
      - Fragment Offset: 308 (= 2464 bytes)
      - More 

In [12]:
# Comprehensive Fragmentation Comparison
from fragmentation_visualizer import FragmentationVisualizer

print("=" * 70)
print("IPv4 vs IPv6 Fragmentation Comparison")
print("=" * 70)

visualizer = FragmentationVisualizer()

# Analyze fragmentation for different payload sizes
print("\n📊 Analyzing fragmentation patterns...")
# Compare protocols first - this populates the data for other methods
visualizer.compare_protocols(payload_sizes=[1000, 2000, 3000, 5000, 8000])

# Generate ASCII charts
print("\n" + "=" * 70)
print("VISUAL COMPARISON - Fragment Count")
print("=" * 70)
visualizer.generate_ascii_chart()

print("\n" + "=" * 70)
print("VISUAL COMPARISON - Efficiency")
print("=" * 70)
visualizer.generate_efficiency_chart()

# Detailed breakdown for a specific payload size
print("\n" + "=" * 70)
print("DETAILED BREAKDOWN - 3000 byte payload")
print("=" * 70)
visualizer.show_detailed_breakdown(3000)

# Export results
print("\n" + "=" * 70)
print("EXPORTING RESULTS")
print("=" * 70)

# Export JSON comparison
visualizer.export_comparison_json("fragmentation_comparison.json")

# Export PCAP files
visualizer.export_pcap("ipv4_comparison.pcap", "ipv6_comparison.pcap")

print("\n" + "=" * 70)
print("KEY OBSERVATIONS")
print("=" * 70)
print("""
1. IPv4 has smaller header (20 bytes) vs IPv6 (40 bytes)
2. IPv6 fragment header adds 8 bytes to each fragment
3. IPv4 uses 16-bit identification field
4. IPv6 uses 32-bit identification field
5. IPv4 can be fragmented by routers (DF flag)
6. IPv6 only source node can fragment (no DF flag)
7. Both protocols have different MTU requirements:
   - IPv4: typical 1500 bytes
   - IPv6: minimum 1280 bytes
""")

IPv4 vs IPv6 Fragmentation Comparison

📊 Analyzing fragmentation patterns...

FRAGMENTATION COMPARISON: IPv4 vs IPv6

IPv4 vs IPv6 Fragmentation Comparison

📊 Analyzing fragmentation patterns...

FRAGMENTATION COMPARISON: IPv4 vs IPv6




Payload    | Protocol | Frags  | Overhead   | Total      | Efficiency  
------------------------------------------------------------------------------------------
1000       | IPv4     | 1      | 20         | 1020       | 98.04       %
           | IPv6     | 1      | 48         | 1048       | 95.42       %
           | Diff     | 0      |        +28 |        +28 |        -2.62%
------------------------------------------------------------------------------------------
2000       | IPv4     | 2      | 40         | 2040       | 98.04       %
           | IPv6     | 2      | 96         | 2096       | 95.42       %

------------------------------------------------------------------------------------------
1000       | IPv4     | 1      | 20         | 1020       | 98.04       %
           | IPv6     | 1      | 48         | 1048       | 95.42       %
           | Diff     | 0      |        +28 |        +28 |        -2.62%
----------------------------------------------------------------------

In [ ]:

pcap_file = "capture.pcap"

if os.path.exists(pcap_file):
    analyzer = FragmentationAnalyzer(pcap_file)
    analyzer.run_full_analysis()
else:
    print(f"⚠️  Fichier exemple non trouvé: {pcap_file}")
    print("\n💡 Pour utiliser ce script:")
    print("   1. Placez vos fichiers .pcap dans le même dossier")
    print("   2. Modifiez la variable 'pcap_file' avec votre nom de fichier")
    print("   3. Ou utilisez l'exemple 2 ci-dessous pour comparer plusieurs fichiers")

# Exemple 2: Comparaison de plusieurs captures
print("\n\n📌 EXEMPLE 2: Comparaison de plusieurs captures")
print("-" * 70)

captures = [
    "ipv4_mtu_1500.pcap",
    "ipv4_mtu_1000.pcap",
    "ipv4_mtu_500.pcap",
    "ipv6_mtu_1500.pcap",
    "ipv6_mtu_1280.pcap"
]

# Vérifie quels fichiers existent
existing_captures = [f for f in captures if os.path.exists(f)]

if existing_captures:
    compare_captures(existing_captures)